<a href="https://colab.research.google.com/github/atstuyuki/yolov5/blob/master/cts_corrdinate_to_polar_20220702.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#nerve_corrdinates.CSVから極座標変換とploar plotする

In [18]:
#CSVファイルをup_load

from google.colab import files
import os
uploaded = files.upload()
#uploadedは辞書型なのでファイル名を取り出し、絶対パスを取得してfullpathという変数を当てる
for k in uploaded.keys():
  print('file name:',k)
fullpath_csv=os.path.abspath(k)
print('fullpath:',fullpath_csv)

Saving nerve_corrdinates (CTS2).csv to nerve_corrdinates (CTS2).csv
file name: nerve_corrdinates (CTS2).csv
fullpath: /content/nerve_corrdinates (CTS2).csv


In [19]:
#CSVファイルの読み込みとデータフレームに変換
import csv
import pandas as pd
df_csv=pd.read_csv(fullpath_csv, header=0,names=['frame_num','x_corr','y_corr'])
df_csv=df_csv.astype(float)
df_csv

,frame_num,x_corr,y_corr
0,1.0,-0.1125,0.012510
1,2.0,-0.3750,-0.074994
2,3.0,-0.3125,-0.037489
3,4.0,-0.5375,-0.074994
4,5.0,-0.8500,-0.124992
...,...,...,...
265,273.0,0.1000,0.025003
266,275.0,0.1125,0.025003
267,277.0,0.1625,0.037505
268,279.0,0.1625,0.037505


#x,y 座標より極座標系に座標変換

In [20]:
import numpy as np
x_cor=df_csv['x_corr']
y_cor=df_csv['y_corr']
#seriesをnumpy.ndarrayに変換
x_cor=x_cor.values
y_cor=y_cor.values
#直交座標を極座標変換
distance=(x_cor**2+y_cor**2)**0.5
rad=np.arctan2(y_cor, x_cor)#引数はy座標 x座標の順 radianを計算
degrees=np.degrees(rad)#角度に戻す

In [21]:
#各フレーム間での移動距離、フレーム毎での移動角度を計算
#1行ずらした行列を作成する
x_cor_1st=x_cor[1:]#最初の1行（0番目）を削除
x_cor_2nd=x_cor[:-1]#最後の1行を削除
y_cor_1st=y_cor[1:]#最初の1行（0番目）を削除
y_cor_2nd=y_cor[:-1]#最後の1行を削除
#ループ処理せず行列ごと演算
delta_distance=((x_cor_1st-x_cor_2nd)**2+(y_cor_1st-y_cor_2nd)**2)**0.5
delta_rad=np.arctan2((y_cor_2nd-y_cor_1st), (x_cor_2nd-x_cor_1st))#引数はy座標 x座標の順
delta_degrees=np.degrees(rad)#degreesというpd.ndarrayが帰ってくる

#横軸に時間軸　縦軸に各フレーム間での移動量をプロット


In [22]:
#@title
#plotlyでグラフを可視化

import plotly
import plotly.graph_objs as go
# Google Colab. やJupyter Lab.でプロットするためには，以下を実行する．
import plotly.io as pio
pio.renderers.default = "colab"
fig = go.Figure() # 1 

fig.add_trace(    #2. グラフを追加・記述する
    go.Scatter(x = df_csv['frame_num'], y = delta_distance,
                  name='delta_distance'))

fig.update_layout( #3. グラフの装飾、制限を追加(titleの表示、x軸の最大値など)
    title = 'nerve movement over time',
    xaxis_title = 'frame number',
    yaxis_title = 'delta distance(mm)',
    showlegend = True
)
fig.show() #4.グラフの描画


#極座標表示　原点（最初のフレーム）からみた極座標

In [23]:
#np.ndarrayをシリーズに変換
distance_s=pd.Series(distance)
degrees_s=pd.Series(degrees)
#SeriesをDataFrameに変換
df_dist=pd.DataFrame(distance_s,columns=['distance'])
df_degrees=pd.DataFrame(degrees_s,columns=['degrees'])
#2つのDataFrameを横結合
df_dist_degrees=pd.concat([df_dist,df_degrees],axis=1)

In [24]:
import plotly.express as px
fig = px.scatter_polar(df_dist_degrees,r='distance',theta='degrees',direction='counterclockwise')
fig.show()

#極座標表示　各フレーム間の移動の極座標

In [25]:
#np.ndarrayをシリーズに変換
delta_distance_s=pd.Series(delta_distance)
delta_degrees_s=pd.Series(delta_degrees)
#SeriesをDataFrameに変換
df_delta_dist=pd.DataFrame(delta_distance_s,columns=['delta_distance'])
df_delta_degrees=pd.DataFrame(delta_degrees_s,columns=['delta_degrees'])
#2つのDataFrameを横結合
df_delta_dist_degrees=pd.concat([df_delta_dist,df_delta_degrees],axis=1)
#Plotly_expessで極座標表示
import plotly.express as px
fig = px.scatter_polar(df_delta_dist_degrees,r='delta_distance',theta='delta_degrees',direction='counterclockwise')
fig.show()

#追加解析したデータを結合してCSV保存
_new.csvとして保存

In [26]:
df_new=pd.concat([df_dist_degrees,df_delta_dist_degrees],axis=1)
df_new=pd.concat([df_csv,df_new],axis=1)
np.savetxt(k+'_new.csv',df_new,delimiter = ',',fmt = '%s')
df_new

,frame_num,x_corr,y_corr,distance,degrees,delta_distance,delta_degrees
0,1.0,-0.1125,0.012510,0.113193,173.654928,0.276700,173.654928
1,2.0,-0.3750,-0.074994,0.382425,-168.690978,0.072889,-168.690978
2,3.0,-0.3125,-0.037489,0.314741,-173.159242,0.228104,-173.159242
3,4.0,-0.5375,-0.074994,0.542706,-172.057176,0.316474,-172.057176
4,5.0,-0.8500,-0.124992,0.859141,-171.634619,0.380171,-171.634619
...,...,...,...,...,...,...,...
265,273.0,0.1000,0.025003,0.103078,14.038023,0.012500,14.038023
266,275.0,0.1125,0.025003,0.115245,12.530409,0.051539,12.530409
267,277.0,0.1625,0.037505,0.166772,12.996274,0.000000,12.996274
268,279.0,0.1625,0.037505,0.166772,12.996274,0.000000,12.996274
